# Lab 2 PySpark:

### Machine Learning

#### But first, let's do some SQL :)

**First, install and import PySPark and SparkSession**

In [1]:
from pyspark.sql import SparkSession

**Now, download the dataset**

In [2]:
!gdown https://drive.google.com/uc?id=1PB6wBDVTM_eocxOyi0lWlLBQOlH0rLe_ -O PatientInfo.csv

/bin/bash: line 1: gdown: command not found


**Create a SparkSession object and name the app "Lab2"**

In [3]:
spark = SparkSession.builder.appName("Lab2").getOrCreate()

**1. Read the file PatientInfo.csv into a dataframe**

In [4]:
df = spark.read.csv("PatientInfo.csv",header=True,inferSchema=True)

**Show the first 5 lines of the dataframe**

In [5]:
from IPython.display import display, HTML
display(HTML("<style>pre {white-space : pre !important ;}</style>"))

In [6]:
df.show(5)

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|         null|released|
|1000000002|  male|30s|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|  Jongno-gu|contact with patient| 2002000001|            17|              null|    2020-01-30|   202

**Now do the same but using SQL select statement**

1. Create a temporary view (table) called patients

In [7]:
df.createOrReplaceTempView("patient")

2. Use SELECT statement to select all columns from the dataframe

In [8]:
spark.sql(""" Select * from patient""").show()

+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|        city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul|  Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|         null|released|
|1000000002|  male|30s|  Korea|   Seoul| Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|   Jongno-gu|contact with patient| 2002000001|            17|              null|    2020-01-30|

3. Limit the output to only 5 rows *using SQL commands*

In [9]:
spark.sql(""" Select * from patient limit 5""").show()

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|         null|released|
|1000000002|  male|30s|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|  Jongno-gu|contact with patient| 2002000001|            17|              null|    2020-01-30|   202

4. Select the count of males and females in the dataset

In [10]:
spark.sql(""" Select sex,count(sex) 
              from patient
              group by sex
              """).show()

+------+----------+
|   sex|count(sex)|
+------+----------+
|  null|         0|
|female|      2218|
|  male|      1825|
+------+----------+



In [11]:
spark.sql(""" Select count(sex) 
              from patient
              """).show()

+----------+
|count(sex)|
+----------+
|      4043|
+----------+



5. Select the count of males and females *as percentage* (how many percent of the data are males and how many are females?)

In [12]:
spark.sql(""" Select sex,(count(sex)/4043 * 100) as percent_of_the_data
              from patient
              group by sex
              """).show()

+------+-------------------+
|   sex|percent_of_the_data|
+------+-------------------+
|  null|                0.0|
|female|  54.86025228790502|
|  male|  45.13974771209498|
+------+-------------------+



6. How many people did survive, and how many didn't?

In [13]:
spark.sql(""" Select state , count(state)
              from patient
              group by state

""").show()

+--------+------------+
|   state|count(state)|
+--------+------------+
|isolated|        2158|
|released|        2929|
|deceased|          78|
+--------+------------+



Now, let's perform some preprocessing using SQL:

1. Convert *age* column to double after removing the 's' at the end -- *hint: check SUBSTRING method*
2. Select only the following columns: `['sex', 'age', 'province', 'state']`
3. Store the result of the query in a new dataframe

In [14]:
spark.sql(""" Select sex, CAST(Substring(age,1,2) as DECIMAL(10,2)) age, province, state
              from patient

""").write.csv("myresult.csv",header=True)

Now view the new dataframe to make sure everything is alright

In [15]:
df_new = spark.read.csv("myresult.csv",header=True,inferSchema=True)
df_new.show()

+------+----+--------+--------+
|   sex| age|province|   state|
+------+----+--------+--------+
|  male|50.0|   Seoul|released|
|  male|30.0|   Seoul|released|
|  male|50.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|female|20.0|   Seoul|released|
|female|50.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|  male|30.0|   Seoul|released|
|female|60.0|   Seoul|released|
|female|50.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|  male|80.0|   Seoul|deceased|
|female|60.0|   Seoul|released|
|  male|70.0|   Seoul|released|
|  male|70.0|   Seoul|released|
|  male|70.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|female|70.0|   Seoul|released|
|female|70.0|   Seoul|released|
+------+----+--------+--------+
only showing top 20 rows



**Now, let's get back to spark operations**

Please copy the following operations from your solution in Lab 1
___

Add a **is_dead** column if patient state is not released then it should yield true, else then False

In [16]:
from pyspark.sql.functions import when, col
df_new = df_new.withColumn("is_dead",when(col("state")=="released",1).otherwise(0))
df_new.show()

+------+----+--------+--------+-------+
|   sex| age|province|   state|is_dead|
+------+----+--------+--------+-------+
|  male|50.0|   Seoul|released|      1|
|  male|30.0|   Seoul|released|      1|
|  male|50.0|   Seoul|released|      1|
|  male|20.0|   Seoul|released|      1|
|female|20.0|   Seoul|released|      1|
|female|50.0|   Seoul|released|      1|
|  male|20.0|   Seoul|released|      1|
|  male|20.0|   Seoul|released|      1|
|  male|30.0|   Seoul|released|      1|
|female|60.0|   Seoul|released|      1|
|female|50.0|   Seoul|released|      1|
|  male|20.0|   Seoul|released|      1|
|  male|80.0|   Seoul|deceased|      0|
|female|60.0|   Seoul|released|      1|
|  male|70.0|   Seoul|released|      1|
|  male|70.0|   Seoul|released|      1|
|  male|70.0|   Seoul|released|      1|
|  male|20.0|   Seoul|released|      1|
|female|70.0|   Seoul|released|      1|
|female|70.0|   Seoul|released|      1|
+------+----+--------+--------+-------+
only showing top 20 rows



**Please split the data into train and test dataframes**

*Ratio: 80:20 - Seed=42*

In [17]:
trainDf, testDf = df_new.randomSplit([.8,.2],seed=42)

In [18]:
testDf.show(5)

+----+----+-----------+--------+-------+
| sex| age|   province|   state|is_dead|
+----+----+-----------+--------+-------+
|null|null|Gyeonggi-do|isolated|      0|
|null|null|Gyeonggi-do|isolated|      0|
|null|null|Gyeonggi-do|isolated|      0|
|null|null|Gyeonggi-do|isolated|      0|
|null|null|Gyeonggi-do|isolated|      0|
+----+----+-----------+--------+-------+
only showing top 5 rows



**Now, let's import RandomForestClassifier and start our ML pipeline**

In [19]:
from pyspark.ml.classification import RandomForestClassifier

**Create a pipeline that contains the following stages:**

- Imputer: impute the null values in `age` column to the mean value
- StringIndexer: convert `sex` to `is_male` and `province` to `province_index` as numerical values
- OneHotEncoder: perform one hot encoding on both `is_male` and -province_index`
- VectorAssembler: assemble feature vector from the following columns: `'age', 'is_male', 'province_index'`
- RandomForestClassifier: final estimator

In [20]:
from pyspark.ml import Pipeline

In [21]:
from pyspark.ml.feature import Imputer, StringIndexer, VectorAssembler
from pyspark.ml.feature import OneHotEncoder
imputer = Imputer(inputCol="age",outputCol="age")

index = StringIndexer(inputCols=["sex","province"], outputCols=["is_male","province_index"],handleInvalid="skip")
ohe = OneHotEncoder(inputCols=["is_male","province_index"], outputCols=["is_male_","province_index_"])
assembler = VectorAssembler(
                            inputCols=["age","is_male_","province_index_"],
                            outputCol="features")
rfc = RandomForestClassifier(featuresCol="features",labelCol="is_dead",predictionCol="prediction")
model = Pipeline(stages=[imputer,index,ohe,assembler,rfc]).fit(trainDf)

Fit the pipeline to the train dataframe

Now transform the test DF to get predictions

In [22]:
pred = model.transform(testDf)

Show the final predictions DF

In [23]:
pred.show()

+------+-----------------+-----------------+--------+-------+-------+--------------+-------------+---------------+--------------------+--------------------+--------------------+----------+
|   sex|              age|         province|   state|is_dead|is_male|province_index|     is_male_|province_index_|            features|       rawPrediction|         probability|prediction|
+------+-----------------+-----------------+--------+-------+-------+--------------+-------------+---------------+--------------------+--------------------+--------------------+----------+
|female|40.83025210084033|Chungcheongnam-do|released|      1|    0.0|           4.0|(1,[0],[1.0])| (16,[4],[1.0])|(18,[0,1,6],[40.8...|[8.13297885295184...|[0.40664894264759...|       1.0|
|female|40.83025210084033|Chungcheongnam-do|released|      1|    0.0|           4.0|(1,[0],[1.0])| (16,[4],[1.0])|(18,[0,1,6],[40.8...|[8.13297885295184...|[0.40664894264759...|       1.0|
|female|40.83025210084033|            Daegu|isolated|  

In [24]:
pred.select('features','is_dead','prediction').show(5,truncate=False)

+----------------------------------------+-------+----------+
|features                                |is_dead|prediction|
+----------------------------------------+-------+----------+
|(18,[0,1,6],[40.83025210084033,1.0,1.0])|1      |1.0       |
|(18,[0,1,6],[40.83025210084033,1.0,1.0])|1      |1.0       |
|(18,[0,1,8],[40.83025210084033,1.0,1.0])|0      |0.0       |
|(18,[0,1,4],[40.83025210084033,1.0,1.0])|0      |0.0       |
|(18,[0,1,3],[40.83025210084033,1.0,1.0])|0      |1.0       |
+----------------------------------------+-------+----------+
only showing top 5 rows



**Model Evaluation**

Now let's evaluate our model! Let's get the accuracy of our model

In [25]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
ev = BinaryClassificationEvaluator(rawPredictionCol="prediction",labelCol="is_dead",metricName="areaUnderPR")

In [26]:
ev.evaluate(pred)

0.8873873074990393

In [27]:
pandsDf = pred.toPandas()

In [28]:
pandsDf

,sex,age,province,state,is_dead,is_male,province_index,is_male_,province_index_,features,rawPrediction,probability,prediction
0,female,40.830252,Chungcheongnam-do,released,1,0.0,4.0,(1.0),"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","(40.83025210084033, 1.0, 0.0, 0.0, 0.0, 0.0, 1...","[8.132978852951844, 11.867021147048156]","[0.4066489426475922, 0.5933510573524078]",1.0
1,female,40.830252,Chungcheongnam-do,released,1,0.0,4.0,(1.0),"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","(40.83025210084033, 1.0, 0.0, 0.0, 0.0, 0.0, 1...","[8.132978852951844, 11.867021147048156]","[0.4066489426475922, 0.5933510573524078]",1.0
2,female,40.830252,Daegu,isolated,0,0.0,6.0,(1.0),"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","(40.83025210084033, 1.0, 0.0, 0.0, 0.0, 0.0, 0...","[15.013567842167967, 4.986432157832032]","[0.7506783921083984, 0.24932160789160157]",0.0
3,female,40.830252,Gyeonggi-do,isolated,0,0.0,2.0,(1.0),"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(40.83025210084033, 1.0, 0.0, 0.0, 1.0, 0.0, 0...","[16.100563485372422, 3.89943651462758]","[0.8050281742686209, 0.19497182573137894]",0.0
4,female,40.830252,Gyeongsangbuk-do,isolated,0,0.0,1.0,(1.0),"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(40.83025210084033, 1.0, 0.0, 1.0, 0.0, 0.0, 0...","[3.4525672833985075, 16.54743271660149]","[0.17262836416992539, 0.8273716358300746]",1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,male,90.000000,Gyeonggi-do,isolated,0,1.0,2.0,(0.0),"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(90.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,...","[16.16880135876239, 3.8311986412376102]","[0.8084400679381195, 0.19155993206188052]",0.0
806,male,90.000000,Gyeongsangbuk-do,deceased,0,1.0,1.0,(0.0),"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(90.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[3.60874068373907, 16.391259316260932]","[0.1804370341869535, 0.8195629658130466]",1.0
807,male,90.000000,Gyeongsangbuk-do,isolated,0,1.0,1.0,(0.0),"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(90.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[3.60874068373907, 16.391259316260932]","[0.1804370341869535, 0.8195629658130466]",1.0
808,male,90.000000,Gyeongsangbuk-do,isolated,0,1.0,1.0,(0.0),"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(90.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[3.60874068373907, 16.391259316260932]","[0.1804370341869535, 0.8195629658130466]",1.0


Excellent! Now let's generate the confusion matrix of our predictions

*Hint: we can use `scikit-learn`'s `classification_report`. You will need to transform the predictions into pandas DF first*

In [31]:
from sklearn.metrics import classification_report
print(classification_report(pandsDf['is_dead'], pandsDf['prediction']))

              precision    recall  f1-score   support

           0       0.91      0.81      0.86       306
           1       0.89      0.95      0.92       504

    accuracy                           0.90       810
   macro avg       0.90      0.88      0.89       810
weighted avg       0.90      0.90      0.90       810

